# GOALS

- *Top hiring companies (curiosity)*
- Positon Categorization (Analyst, Scientist & Engineer)
- Proportion Data vs Science vs Engineer (Roles)
- Modality Proportion (Trainee, Junior, Mid & Senior)
- Seniority x Category
- Location Percentage
- Keywords Exploration (soft, hard and tools)

# LIBRARIES

In [244]:
import re
import sys
import nltk
import unidecode
import numpy as np
import pandas as pd
import seaborn as sns
from nltk import Text
from textblob import TextBlob
from langdetect import detect
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords
from googletrans import Translator
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline
pd.set_option('display.max_colwidth', None)

In [245]:
df = pd.read_csv('data_jobs_total.csv')

In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        383 non-null    object
 1   company      382 non-null    object
 2   location     382 non-null    object
 3   modality     371 non-null    object
 4   description  382 non-null    object
dtypes: object(5)
memory usage: 15.1+ KB


In [247]:
df

title  \
0                                                                       Senior Game Data Analyst   
1                                                                 Data Analyst (Operations Team)   
2                                                                                  Data Engineer   
3                                                                       Analista de Datos Junior   
4                                                                                   Data Analyst   
..                                                                                           ...   
378                                                      Tech & Data Strategy Consultant (f/m/d)   
379  Ayudante de investigación para el grupo de investigación AI and Data for Society (Temporal)   
380                                                        Consultant (all genders) Data Systems   
381                                                              Product Data Consultant (m/f/d)   
382                                                                     Principal Data Scientist   

                             company                     location modality  \
0                      Tilting Point  Barcelona, Catalonia, Spain   Hybrid   
1                            Holaluz  Barcelona, Catalonia, Spain  On-site   
2                              IOMED  Barcelona, Catalonia, Spain   Hybrid   
3                                CGI                        Spain   Remote   
4                        Olala Homes  Barcelona, Catalonia, Spain   Hybrid   
..                               ...                          ...      ...   
378              Omnicom Media Group  Barcelona, Catalonia, Spain   Hybrid   
379  Universitat Oberta de Catalunya  Barcelona, Catalonia, Spain   Hybrid   
380                        INVENSITY  Barcelona, Catalonia, Spain  On-site   
381                         ZF Group  Barcelona, Catalonia, Spain   Hybrid   
382                          Experfy  Barcelona, Catalonia, Spain  On-site   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

# Let's remove duplicates

In [248]:
df = df.drop_duplicates()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 383 entries, 0 to 382
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        383 non-null    object
 1   company      382 non-null    object
 2   location     382 non-null    object
 3   modality     371 non-null    object
 4   description  382 non-null    object
dtypes: object(5)
memory usage: 18.0+ KB


# Positions' Categorization

In [268]:
df['title'] = df['title'].str.lower()

analysts = len(df[df['title'].str.contains("anal")])
scientists = len(df[df['title'].str.contains("scien")])
engineers = len(df[df['title'].str.contains("engin")])

scientists += len(df[df['title'].str.contains("cient")])
engineers += len(df[df['title'].str.contains("ingen")])

print(f'Analysts: {round(analysts/df.shape[0]*100)}%')
print(f'Scientists: {round(scientists/df.shape[0]*100)}%')
print(f'Engineers: {round(engineers/df.shape[0]*100)}%')

Analysts: 29%
Scientists: 40%
Engineers: 24%


In [269]:
n = 0
overlapped = []
for index, job in df['title'].iteritems():
    if 'scien' in job and 'anal' in job:
        n += 1
        overlapped.append(index)
per = round(n/(analysts+scientists)*100)
print(f'{n} job positions are overlapped, {per}% of all Data Analyst, Data Engineer & Data Scientist positions.')

0 job positions are overlapped, 0% of all Data Analyst, Data Engineer & Data Scientist positions.


In [271]:
df.iloc[overlapped, 0]


Series([], Name: title, dtype: object)

In [272]:
df.iloc[84, 0] = 'data scientist'


In [273]:
def find_missing_titles(df):
    missing_titles = []
    for title in df['title']:
        if 'anal' not in title and 'scien' not in title and 'engin' not in title:
            missing_titles.append(title)
    return missing_titles

In [274]:
missing_titles = find_missing_titles(df)
for title in missing_titles:
    print(title)

data manager
científico de datos - ia
data operations intern
junior data & business intelligence support specialist
técnico/a de base de datos
database administrator
data consultant
data manager
especialista en bbdd y modelado de datos
data leader (sql - big data)
data quality manager (barcelona)
internship programme data (programa de becas)
senior data consultant
lead data platform
director of data
transformation & data associate
data acquisition strategist (m/f/d)
azure data consultant
data modelling architect
🏠 arquitecto de datos dwh - datamart 🏠
azure data consultant
senior data architect
data specialist
data architect - digital markets
data architect
data manager & statistician, malaria immunology group (short term)
principal data architect & data steward
data manager for the ubioesdm
head of global master data
data manager strategy
database administrator (dba) - remote within spain
data & transformation office leader
data coordinator (fixed term)
product data consultant (m/f/d)


# Let's categorize: Analyst, Scientist and Engineer positions

In [275]:
category = []
dropped = []
for index, position in df['title'].iteritems():
    if "scien" in position:
        category.append('data scientist')
    elif "anal" in position:
        category.append('data analyst')
    elif "engin" in position:
        category.append('data engineer')
    elif "ingeni" in position:
        category.append('data engineer')
    elif "cienti" in position:
        category.append('data scientist')
    else:
        dropped.append(index)

df.drop(dropped, inplace=True)
df.reset_index(inplace=True, drop=True)
df['position'] = category

In [276]:
df

title               company  \
0                senior game data analyst         Tilting Point   
1          data analyst (operations team)               Holaluz   
2                           data engineer                 IOMED   
3                analista de datos junior                   CGI   
4                            data analyst           Olala Homes   
..                                    ...                   ...   
278           data science oncology r & d  Morgan Philips Group   
279                         data engineer   HOLA CONSULTORES SL   
280  senior data engineer - data platform             Rover.com   
281       cloud engineer (data analytics)      Zurich Insurance   
282              principal data scientist               Experfy   

                        location modality  \
0    Barcelona, Catalonia, Spain   Hybrid   
1    Barcelona, Catalonia, Spain  On-site   
2    Barcelona, Catalonia, Spain   Hybrid   
3                          Spain   Remote   
4    Barcelona, Catalonia, Spain   Hybrid   
..                           ...      ...   
278  Barcelona, Catalonia, Spain  On-site   
279  Barcelona, Catalonia, Spain  On-site   
280  Barcelona, Catalonia, Spain   Hybrid   
281  Barcelona, Catalonia, Spain   Hybrid   
282  Barcelona, Catalonia, Spain  On-site   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

# Data Exploration

In [277]:
df['position'].value_counts()

data analyst      112
data engineer      87
data scientist     84
Name: position, dtype: int64

In [278]:
print(f"Analysts: {round(len(df[df['position'] == 'data analyst'])/df.shape[0]*100)}%")
print(f"Scientists: {round(len(df[df['position'] == 'data scientist'])/df.shape[0]*100)}%")
print(f"Engineer: {round(len(df[df['position'] == 'data engineer'])/df.shape[0]*100)}%")


Analysts: 40%
Scientists: 30%
Engineer: 31%


# Top companies most looking for talent

In [279]:
df['company'].value_counts()


Glovo                  13
eDreams ODIGEO         10
HP                      9
Zurich Insurance        8
AstraZeneca             7
                       ..
Logicalis Spain         1
Moody's Corporation     1
Q-tech                  1
Gauss & Neumann         1
Rover.com               1
Name: company, Length: 152, dtype: int64

# Modality Exploration

In [281]:
df['modality'].value_counts()

Hybrid     156
On-site     53
Remote      38
Unknown     26
Name: modality, dtype: int64

# Location Exploration

In [282]:
df['location'].value_counts()


Barcelona, Catalonia, Spain                    220
Greater Barcelona Metropolitan Area             21
Spain                                           17
Sant Cugat del Vallès, Catalonia, Spain         14
Palau-solità i Plegamans, Catalonia, Spain       4
L'Hospitalet de Llobregat, Catalonia, Spain      2
Esplugues de Llobregat, Catalonia, Spain         2
Cornellà de Llobregat, Catalonia, Spain          1
Mataró, Catalonia, Spain                         1
Name: location, dtype: int64

In [286]:
df['location'].value_counts()


Barcelona, Catalonia, Spain                    220
Greater Barcelona Metropolitan Area             21
Spain                                           17
Sant Cugat del Vallès, Catalonia, Spain         14
Palau-solità i Plegamans, Catalonia, Spain       4
L'Hospitalet de Llobregat, Catalonia, Spain      2
Esplugues de Llobregat, Catalonia, Spain         2
Cornellà de Llobregat, Catalonia, Spain          1
Mataró, Catalonia, Spain                         1
Name: location, dtype: int64

# Cleaning Descriptions

In [287]:
# Replace - jumps with spaces
df['description'] = df['description'].str.replace('\n', ' ')
# Delete the hyphens & ¿
df['description'] = df['description'].map(lambda x: x.replace('-', '').replace('¿', ''))
# Normalise words (no accents, etc.)
df['description'] = df['description'].map(lambda x: unidecode.unidecode(x))
# Adding spaces in a word if it contains a capitalised letter in between
df['description'] = df['description'].map(lambda x: re.sub(r"(?<![A-Z])(?<!^)([A-Z])",r" \1", x))
# Adding spaces before and after numbers
df['description'] = df['description'].map(lambda x: re.sub(r"([0-9]+(\.[0-9]+)?)", r" \1", x))
# Substitute n number of spaces by just one space
df['description'] = df['description'].map(lambda x: ' '.join(x.split()))
# Remove space before dots and after opening parenthesis
df['description'] = df['description'].map(lambda x: x.replace('( ', '(').replace(' .', '.').replace('/ ', '/'))
# Fix problems with the normalisation
df['description'] = df['description'].map(lambda x: x.replace('ano', 'año').replace('anos', 'años'))


AttributeError: 'float' object has no attribute 'replace'

In [150]:
df.head(2)

title        company                     location  \
0        senior game data analyst  Tilting Point  Barcelona, Catalonia, Spain   
1  data analyst (operations team)        Holaluz  Barcelona, Catalonia, Spain   

  modality  \
0   Hybrid   
1  On-site   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [151]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import spacy
nltk.download()

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Descargar modelo de SpaCy
spacy.cli.download("es_core_news_sm")

# Cargar modelo de SpaCy
nlp = spacy.load("es_core_news_sm")

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 6.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


- Let's convert list to string ('Description' column)

In [152]:
tokenizer = RegexpTokenizer(r'\w+')
df['description'] = df['description'].map(lambda x: tokenizer.tokenize(x.lower()))
# Removing stopwords
stop_words = stopwords.words('english')
#for index, des in df['description'].iteritems():
    #df.iloc[index, 3] =  [w for w in des if not w in stop_words]
# Lemmatization
def lemma(text):
    return [WordNetLemmatizer().lemmatize(w) for w in text]
df['description'] = df['description'].apply(lemma)



/var/folders/q7/v3v0dp3x75b1z141v2hlcr3c0000gn/T/ipykernel_35550/868813695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = df['description'].map(lambda x: tokenizer.tokenize(x.lower()))
/var/folders/q7/v3v0dp3x75b1z141v2hlcr3c0000gn/T/ipykernel_35550/868813695.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = df['description'].apply(lemma)


In [153]:
df.head(2)

,title,company,location,modality,description,position
0,senior game data analyst,Tilting Point,"Barcelona, Catalonia, Spain",Hybrid,"[at, tilting, point, we, combine, data, technology, talent, and, groundbreaking, marketing, to, scale, game, to, success, a, the, senior, game, data, analyst, you, will, work, closely, with, game, team, to, help, inform, decisionmaking, regarding, feature, of, the, game, balancing, player, interaction, finding, hidden, opportunity, and, much, more, your, future, at, tilting, point, analyze, large, complex, datasets, to, address, adhoc, and, longterm, operational, and, strategic, business, question, draw, meaningful, conclusion, and, provide, actionable, insight, from, data, to, shape, and, influence, ingame, feature, economy, and, future, product, roadmaps, own, the, taxonomy, for, game, data, ensuring, all, key, performance, metric, are, measurable, and, ...]",data analyst
1,data analyst (operations team),Holaluz,"Barcelona, Catalonia, Spain",On-site,"[description, quienes, somos, somos, holaluz, la, empresa, que, lleva, una, decada, revolucionando, el, sector, energetico, nacimos, para, cambiar, el, mundo, conectando, a, la, persona, con, la, energia, verde, y, tu, y, el, planeta, sois, nuestra, motivacion, para, ser, como, somos, y, hacer, lo, que, hacemos, nuestra, voluntad, de, hacer, la, cosas, de, un, modo, distinto, no, ha, llevado, a, liderar, la, transformacion, del, sector, electrico, y, a, ser, la, primera, electrica, en, obtener, el, certificado, bcorp, te, sumas, unete, a, la, revolucion, que, haras, formaras, parte, del, chapter, de, data, analytics, con, foco, en, el, area, de, operaciones, que, esperamos, ...]",data analyst


# NLTK - Descriptions and titles

In [154]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')









[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Descriptions

In [155]:
all_des = []
all_wrds = []
for des in df['description']:
    all_des.append(des)
    for word in des:
        all_wrds.append(word)
all_wrds = [w.replace('visualisation', 'visualization') for w in all_wrds]
all_wrds = [w.replace('modelling', 'modeling') for w in all_wrds]
all_wrds = [w.replace('artificial intelligence', 'ai') for w in all_wrds]
all_wrds = [w.replace('dashboarding', 'dashboard') for w in all_wrds]
all_wrds = [w.replace('kpis', 'kpi') for w in all_wrds]
all_wrds = [w.replace('report', 'reporting') for w in all_wrds]
all_wrds = [w.replace('predict', 'prediction') for w in all_wrds]
all_wrds = [w.replace('creative', 'creativity') for w in all_wrds]
for l in all_wrds:
    if l == 'team' and all_wrds[all_wrds.index(l) + 1] == 'working':
        all_wrds[all_wrds.index(l) + 1] = 'work'
descriptions = Text(all_wrds)

In [156]:
all_des_a = []
all_des_s = []
all_des_e = []
for des in df[df['position'] == 'data analyst']['description']:
    for word in des:
        all_des_a.append(word)
for des in df[df['position'] == 'data scientist']['description']:
    for word in des:
        all_des_s.append(word)
for des in df[df['position'] == 'data engineer']['description']:
    for word in des:
        all_des_e.append(word)



descriptions_a = Text(all_des_a)
descriptions_s = Text(all_des_s)
descriptions_e = Text(all_des_e)

titles

In [157]:
all_titles = []
for title in df['title']:
    for word in title:
        all_titles.append(word)
titles = Text(all_titles)

In [158]:
for index, d in df['description'].iteritems():
    d = [w.replace('visualisation', 'visualization') for w in d]
    d = [w.replace('modelling', 'modeling') for w in d]
    d = [w.replace('artificial intelligence', 'ai') for w in d]
    d = [w.replace('dashboarding', 'dashboard') for w in d]
    d = [w.replace('kpis', 'kpi') for w in d]
    d = [w.replace('report', 'reporting') for w in d]
    d = [w.replace('predict', 'prediction') for w in d]
    d = [w.replace('creative', 'creativity') for w in d]
    df['description'][index] = d

/var/folders/q7/v3v0dp3x75b1z141v2hlcr3c0000gn/T/ipykernel_35550/3170481636.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'][index] = d


# EXPERIENCE EXPLORATION

In [161]:
#In the title
seniority = 0
juniority = 0

for des in df['title']:
    if 'senior' in des or 'sr' in des:
        seniority += 1
    elif 'junior' in des or 'jr' in des or 'intern' in des:
        juniority += 1

print(seniority)
print(juniority)

48
11


In [162]:
#In the descrption
seniority = 0
juniority = 0

for des in df['description']:
    if 'senior' in des or 'sr' in des:
        seniority += 1
    elif 'junior' in des or 'jr' in des or 'intern' in des:
        juniority += 1

print(seniority)
print(juniority)


47
10


# Lets segment it by experience

In [165]:
seniority = []
for des in df['description']:
    if 'senior' in des or 'sr' in des:
        seniority.append('senior')
    elif 'junior' in des:
        seniority.append('junior')
    else:
        try:
            s = Text(des).concordance_list('year')[0]
            try:
                y = int(s[0][-1])
                if y < 4:
                    seniority.append('junior')
                else:
                    seniority.append('senior')
            except ValueError:
                seniority.append('unknown')
        except IndexError:
            seniority.append('unknown')

In [166]:
df['seniority'] = seniority
df['seniority'].value_counts()

/var/folders/q7/v3v0dp3x75b1z141v2hlcr3c0000gn/T/ipykernel_35550/1420023064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['seniority'] = seniority


unknown    95
senior     75
junior     30
Name: seniority, dtype: int64

# Keywords (one of the most interesting parts)

TOOLS

In [167]:
tools = ['excel','qlik', 'python', 'r', 'sql', 'git', 'power bi', 'aws', 'azure', 'looker',
        'agile', 'knime', 'matlab', 'tableau']

HARD SKILLS

In [168]:
hard_skills = ['programming', 'machine learning','visualization',
               'modeling','business analyst','research','deep learning', 'analytical skill',
               'optimization', 'automation', 'ab', 'ai', 'dashboard',
               'kpi', 'big data', 'data mining', 'etl', 'reporting', 'segmentation',
               'business intelligence', 'natural language', 'prediction', 'algorithm']

SOFT SKILLS

In [169]:
soft_skills = ['communication', 'client oriented', 'team work', 'presentation', 'management', 'decision making',
              'creativity', 'curiosity', 'leadership', 'problem solving', 'attention detail',
              'motivation', 'proactive', 'empathetic', ]

# EXTRACTION - KW

In [170]:
tfidf = TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w+\\b')
text = [' '.join(doc) for doc in df['description']]
tfidf.fit(text)
dtm = tfidf.transform(text)
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names_out())

In [171]:
key_words = dtm.loc[:, dtm.columns.isin(tools + hard_skills + soft_skills)]


# KEYWORDS' DF

In [172]:
df_key = pd.concat([df, key_words], axis=1)


In [173]:
df_key.drop(['description', 'title'], axis=1, inplace=True)


In [174]:
df_key.groupby('position').agg('mean')


,ab,agile,ai,algorithm,analytical skill,automation,aws,azure,big data,business analyst,...,python,qlik,r,reporting,research,segmentation,sql,tableau,team work,visualization
position,,,,,,,,,,,,,,,,,,,,,
data analyst,0.000693,0.002133,0.002794,0.000659,0.001844,0.003708,0.001177,0.003497,0.001957,0.000455,...,0.004932,0.001482,0.006205,0.009537,0.002993,0.000567,0.009209,0.008114,0.001108,0.009264
data engineer,0.000000,0.006779,0.006888,0.001659,0.000854,0.004842,0.009280,0.011478,0.007821,0.000671,...,0.008346,0.000920,0.002862,0.002993,0.002555,0.000000,0.012958,0.001220,0.001751,0.002555
data scientist,0.000815,0.003811,0.010927,0.012848,0.000000,0.001596,0.006608,0.002517,0.007143,0.000711,...,0.008883,0.001232,0.010254,0.003525,0.009485,0.005103,0.006424,0.002364,0.000000,0.008003


In [175]:
df_key.to_csv('df_keys.csv', index=None, header=True)


In [176]:
test = pd.read_csv('df_keys.csv')
test

,company,location,modality,position,seniority,ab,agile,ai,algorithm,analytical skill,...,python,qlik,r,reporting,research,segmentation,sql,tableau,team work,visualization
0,Tilting Point,"Barcelona, Catalonia, Spain",Hybrid,data analyst,senior,0.0,0.00000,0.000000,0.000000,0.00000,...,0.012842,0.0,0.021384,0.000000,0.000000,0.000000,0.013751,0.023149,0.0,0.020316
1,Holaluz,"Barcelona, Catalonia, Spain",On-site,data analyst,unknown,0.0,0.00000,0.000000,0.000000,0.00000,...,0.002660,0.0,0.000000,0.000000,0.000000,0.000000,0.002849,0.000000,0.0,0.000000
2,IOMED,"Barcelona, Catalonia, Spain",Hybrid,data engineer,junior,0.0,0.00000,0.029272,0.000000,0.00000,...,0.014710,0.0,0.000000,0.000000,0.028991,0.000000,0.023627,0.000000,0.0,0.000000
3,CGI,Spain,Remote,data analyst,junior,0.0,0.00000,0.000000,0.000000,0.00000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Olala Homes,"Barcelona, Catalonia, Spain",Hybrid,data analyst,junior,0.0,0.00000,0.000000,0.000000,0.02439,...,0.000000,0.0,0.000000,0.041914,0.000000,0.022681,0.008438,0.000000,0.0,0.037398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Agoda,"Barcelona, Catalonia, Spain",Unknown,data engineer,senior,0.0,0.01013,0.000000,0.000000,0.00000,...,0.017280,0.0,0.000000,0.000000,0.000000,0.000000,0.012336,0.000000,0.0,0.000000
196,AstraZeneca,"Barcelona, Catalonia, Spain",Unknown,data scientist,senior,0.0,0.00000,0.029870,0.015711,0.00000,...,0.007505,0.0,0.000000,0.000000,0.029584,0.000000,0.000000,0.000000,0.0,0.011874
197,AstraZeneca,"Barcelona, Catalonia, Spain",Unknown,data scientist,senior,0.0,0.00000,0.029870,0.015711,0.00000,...,0.007505,0.0,0.000000,0.000000,0.029584,0.000000,0.000000,0.000000,0.0,0.011874
198,AstraZeneca,"Barcelona, Catalonia, Spain",Unknown,data scientist,unknown,0.0,0.00000,0.000000,0.000000,0.00000,...,0.006194,0.0,0.020627,0.000000,0.024413,0.000000,0.000000,0.000000,0.0,0.000000
